## OPTIMIZATION CASE STUDY
### BUSINESS ANALYTICS,DS C17 BATCH 2

In [1]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [2]:
from __future__ import division
from pyomo.opt import SolverFactory

In [3]:
# Reading the data from Excel workbook
demanddist = pd.read_excel("Staffing+Data.xlsx", sheet_name = "DemandData")
StaffAv = pd.read_excel("Staffing+Data.xlsx", sheet_name = "StaffAvailability")
Cost = pd.read_excel("Staffing+Data.xlsx", sheet_name = "Cost")
transactioncost = pd.read_excel("Staffing+Data.xlsx", sheet_name = "ServiceRate")

## Data pre-processing 

In [4]:
# Create the required Python data structures for indexes and parameters
# Demand Data
demand = demanddist.set_index(['State','Month'])['Demand'].to_dict()

# FTE Salary
FTE_Salary = Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()

# Unit Outsourcing Cost
UnitOutSourceCost = Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()


#No.of applications when an FTE is working full-time
FTE_AppServeRate = transactioncost.iloc[0,0]

# Staff Availability for serving the insurance applications
StaffAv_LB = StaffAv.set_index(['State','Month'])['LB'].to_dict()
StaffAv_UB = StaffAv.set_index(['State','Month'])['UB'].to_dict()
StaffAvPer = StaffAv.set_index(['State','Month'])['StaffAvPer'].to_dict()

#Index
BankLoc = Cost['State'].unique()
Month = Cost['Month'].unique()

In [5]:
Bankloc=list(BankLoc)
Month= list(Month)

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [6]:
# Creating a model instance
model = ConcreteModel()

In [7]:
# Define Pyomo sets and Parameters
model.BankLoc = Set(initialize=BankLoc)
model.Month = Set(initialize=Month)
#parameter
model.demand = Param(model.BankLoc, model.Month, initialize = demand)

In [8]:
# Decision variables
model.x = Var(BankLoc, Month, within=NonNegativeReals)
model.y = Var(BankLoc, Month, within=NonNegativeIntegers)

### We have two decision variables here: one(x) for the FTE and the other(y) for the number of outsourced applications. In all the questions from here except the last one(question 6), we will define these same set of decision variables. 

In [9]:
# Constraints

## Demand Constraint
        
def demand_constraint(model,b,m):
    return((model.x[b,m]*StaffAvPer[b,m]*40 + model.y[b,m]) == demand[b,m])

model.dmnd = Constraint(BankLoc, Month, rule=demand_constraint)

## Regulatory Restriction
def outsource_constraint_a(model, b, m):
    return(model.y[b,m] <= 0.3*demand[b,m])

model.outsrc_a = Constraint(BankLoc[0], Month, rule=outsource_constraint_a)

def outsource_constraint_b(model, b, m):
    return(model.y[b,m] <= 0.4*demand[b,m])

model.outsrc_b = Constraint(BankLoc[1], Month, rule=outsource_constraint_b)

### Talking about the constraints, we have two constraints here: the demand constraint which says that the demand per state per month must be met nad the second one is the regulatory constraint which says that States A and B cannot outsource more than 30% and 40% of the applications respectively.The demand constraint will remain same throughout this exercise.

In [10]:
# Objective function

model.value = Objective(expr = sum(sum(FTE_Salary[b,m]*model.x[b,m] + UnitOutSourceCost[b,m]*model.y[b,m]
                                       for m in Month)for b in BankLoc), sense= minimize)

### Out objective is to minimize the cost of approving applications by distributing the right number of applications between the FTE's and the vendors while meeting the demand for each state for each month which is covered in the constraint. Our objective function will remain almost same throughout except the case where there will be a 20% increase in outsourcing cost.

In [11]:
# Invoking the solver
result1 = SolverFactory('glpk').solve(model)
result1.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.1636500358581543
# ----------------------------------------------------------
#   Solution Information
# -------------

In [12]:
# Print the value of the objective function
model.pprint()

18 Set Declarations
    BankLoc : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    Month : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'}
    demand_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain        : Size : Members
        None :     2 : BankLoc*Month :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B', 'Mar'), ('B', 'Apr'), ('B', 'May'), ('B', 'Jun'), ('B', 'Jul'), ('B', 'Aug'), ('B', 'Sep'), ('B', 'Oct'), ('B', 'Nov'), ('B', 'Dec'), ('C', 'Jan'), ('C', 'Feb'), ('C', 'Mar'), ('C', 'Apr'), ('C', 'May'), ('C', 'Jun'), ('C', 'Jul'), ('C', 'Aug'), ('C'

In [13]:
model.value()

17962336.448769882

#### The company has to spend $17.9m for the loan approval process

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [14]:
# Creating dataframe for the results
#Creating an empty list
Output = []

for b in BankLoc:
    for m in Month:
        # Full time employees
        FTE=model.x[b,m].value
        
        #No.of outsourced applications
        Outsourced = model.y[b,m].value
        
        Output.append([b,m,FTE,Outsourced])
print(Output)

[['A', 'Jan', 161.728395061728, 0.0], ['A', 'Feb', 160.460526315789, 0.0], ['A', 'Mar', 198.066666666667, 0.0], ['A', 'Apr', 71.78125, 0.0], ['A', 'May', 63.8461538461538, 0.0], ['A', 'Jun', 77.9109589041096, 0.0], ['A', 'Jul', 137.279411764706, 1600.0], ['A', 'Aug', 110.888157894737, 0.0], ['A', 'Sep', 116.018518518519, 0.0], ['A', 'Oct', 120.856164383562, 0.0], ['A', 'Nov', 110.257352941176, 1285.0], ['A', 'Dec', 139.576923076923, 1554.0], ['B', 'Jan', 152.067901234568, 0.0], ['B', 'Feb', 51.875, 1051.0], ['B', 'Mar', 59.5, 1189.0], ['B', 'Apr', 73.0625, 0.0], ['B', 'May', 128.846153846154, 0.0], ['B', 'Jun', 64.6917808219178, 1258.0], ['B', 'Jul', 94.2279411764706, 1708.0], ['B', 'Aug', 51.7105263157895, 1048.0], ['B', 'Sep', 139.413580246914, 0.0], ['B', 'Oct', 85.3767123287671, 1662.0], ['B', 'Nov', 69.2279411764706, 1254.0], ['B', 'Dec', 97.5769230769231, 1690.0], ['C', 'Jan', 35.8641975308642, 0.0], ['C', 'Feb', 64.7039473684211, 0.0], ['C', 'Mar', 63.2666666666667, 0.0], ['C', 

In [15]:
#Converting a 2D list into a dataframe
Output = pd.DataFrame(Output,columns=['State', 'Month','Full-Time Equivalents','Outsourced Applications'])

In [16]:
Output.head()

,State,Month,Full-Time Equivalents,Outsourced Applications
0,A,Jan,161.728395,0.0
1,A,Feb,160.460526,0.0
2,A,Mar,198.066667,0.0
3,A,Apr,71.781250,0.0
4,A,May,63.846154,0.0


In [17]:
# Importing the required library
#from openpyxl import load_workbook

##### We are commenting out the process to write to Excel as we already have the desired results and having this might result in error in the code when someone else runs it from their end

In [18]:
# Writing the results in to an Excel sheet
#book = load_workbook("Output_data.xlsx")

# create excel writer object
#writer = pd.ExcelWriter("Output_data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
#writer.book = book


# write dataframe to excel sheet named 'output'
#Output.to_excel(writer, sheet_name='Qsn2')


# save the excel file
#writer.save()
#print('DataFrame is written successfully to Excel Sheet.')

# Question 3 

#### Expected Scenarios

The company wants to get a solution for the scenario, where no more than 10% of the total applications for every month in each state is handled by the external resources, for quality assurance reasons

- **`Q 3.1-`** What is the additional average cost per application of keeping the 10% limit on outsourcing? 
Note: Compare with the cost in question 2

- **`Q 3.2-`** If the company increases the outsourcing costs by 20% to reflect the cost of quality assurance, what would you recommend the optimal percent of outsourcing should be?



#### Expected output: 
`Subtask 3.1-` Create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the additional average cost per application.

`Subtask 3.2-` Create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the additional average cost per application. Also, print the overall average optimal per cent of outsourcing.

### 3.1 Additional cost per application of keeping the 10% limit on outsourcing

In [19]:
# Creating a model instance
model2 = ConcreteModel()

In [20]:
# Define Pyomo sets and Parameters
#Pyomo sets
model2.BankLoc = Set(initialize=BankLoc)
model2.Month = Set(initialize=Month)
#parameter
model2.demand = Param(model2.BankLoc, model2.Month, initialize = demand)

In [21]:
# Decision variables
model2.x = Var(BankLoc, Month, within=NonNegativeReals)
model2.y = Var(BankLoc, Month, within=NonNegativeIntegers)

In [22]:
# Constraints

## Demand Constraint
        
def demand_constraint(model2,b,m):
    return((model2.x[b,m]*StaffAvPer[b,m]*40 + model2.y[b,m]) == demand[b,m])

model2.dmnd = Constraint(BankLoc, Month, rule=demand_constraint)



#Outsource Constraint
def outsource_constraint(model2,b,m):
    return(model2.y[b,m] <= 0.1*demand[b,m])
model2.outsrc = Constraint(Bankloc,Month, rule = outsource_constraint)

### Here the outsource constraint changes a bit as there cannot be more than 10% of the demand that can be outsourced 

In [23]:
# Objective function
model2.value = Objective(expr = sum(sum(FTE_Salary[b,m]*model2.x[b,m] + UnitOutSourceCost[b,m]*model2.y[b,m]
                                       for m in Month)for b in BankLoc), sense= minimize)

In [24]:
# Invoking the solver
result = SolverFactory('glpk').solve(model2)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 18100979.3212198
  Upper bound: 18100979.3212198
  Number of objectives: 1
  Number of constraints: 73
  Number of variables: 73
  Number of nonzeros: 109
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.15957164764404297
# ----------------------------------------------------------
#   Solution Information
# -----------

In [25]:
# Print the value of the objective function
model2.value()

18100979.321219858

#### The company has to spend $18.1 m 

**`Checkpoint 2:`** The company has to spend around 18.1 m$ in total for the application approval process if there is 10% limit on the outsourcing.

In [26]:
# Creating dataframe for the results
Output_outsource_const = []

for b in BankLoc:
    for m in Month:
        # Full time employees
        FTE=model2.x[b,m].value
        
        #No.of outsourced applications
        Outsourced = model2.y[b,m].value
        
        Output_outsource_const.append([b,m,FTE,Outsourced])
print(Output_outsource_const)

Output_outsource_const = pd.DataFrame(Output_outsource_const,columns=['State', 'Month','Full-Time Equivalents','Outsourced Applications'])
Output_outsource_const.head()

[['A', 'Jan', 161.728395061728, 0.0], ['A', 'Feb', 160.460526315789, 0.0], ['A', 'Mar', 198.066666666667, 0.0], ['A', 'Apr', 71.78125, 0.0], ['A', 'May', 63.8461538461538, 0.0], ['A', 'Jun', 77.9109589041096, 0.0], ['A', 'Jul', 176.507352941176, 533.0], ['A', 'Aug', 110.888157894737, 0.0], ['A', 'Sep', 116.018518518519, 0.0], ['A', 'Oct', 120.856164383562, 0.0], ['A', 'Nov', 141.764705882353, 428.0], ['A', 'Dec', 179.423076923077, 518.0], ['B', 'Jan', 152.067901234568, 0.0], ['B', 'Feb', 77.8289473684211, 262.0], ['B', 'Mar', 89.2333333333333, 297.0], ['B', 'Apr', 73.0625, 0.0], ['B', 'May', 128.846153846154, 0.0], ['B', 'Jun', 97.0205479452055, 314.0], ['B', 'Jul', 141.323529411765, 427.0], ['B', 'Aug', 77.5657894736842, 262.0], ['B', 'Sep', 139.413580246914, 0.0], ['B', 'Oct', 128.082191780822, 415.0], ['B', 'Nov', 103.823529411765, 313.0], ['B', 'Dec', 146.346153846154, 422.0], ['C', 'Jan', 35.8641975308642, 0.0], ['C', 'Feb', 64.7039473684211, 0.0], ['C', 'Mar', 63.2666666666667, 0

,State,Month,Full-Time Equivalents,Outsourced Applications
0,A,Jan,161.728395,0.0
1,A,Feb,160.460526,0.0
2,A,Mar,198.066667,0.0
3,A,Apr,71.781250,0.0
4,A,May,63.846154,0.0


In [27]:
# Writing the results in to an Excel sheet
#book = load_workbook("Output_data.xlsx")

# create excel writer object
#writer = pd.ExcelWriter("Output_data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
#writer.book = book


# write dataframe to excel sheet named 'output'
#Output_outsource_const.to_excel(writer, sheet_name='Qsn_3_1')


# save the excel file
#writer.save()
#print('DataFrame is written successfully to Excel Sheet.')

In [28]:
# Additional cost per application
print((model2.value()-model.value())/sum(demand.values()))

1.223744174007233


### Additional cost per application is $1.22

### 3.2  Increase in the outsourcing costs by 20% 

In [29]:
# Creating a model instance
model3 = ConcreteModel()

In [30]:
# Define Pyomo sets and Parameters
#Pyomo sets
model3.BankLoc = Set(initialize=BankLoc)
model3.Month = Set(initialize=Month)
#parameter
model3.demand = Param(model3.BankLoc, model3.Month, initialize = demand)

In [31]:
# Decision variables
model3.x = Var(BankLoc, Month, within=NonNegativeReals)
model3.y = Var(BankLoc, Month, within=NonNegativeIntegers)

In [32]:
# Constraints

## Demand Constraint
def demand_constraint(model3,b,m):
    return((model3.x[b,m]*StaffAvPer[b,m]*40 + model3.y[b,m]) == demand[b,m])

model3.dmnd = Constraint(BankLoc, Month, rule=demand_constraint)



#Outsource Constraint
def outsource_constraint(model3,b,m):
    return(model3.y[b,m] <= 0.1*demand[b,m])
model3.outsrc = Constraint(Bankloc,Month, rule = outsource_constraint)

In [33]:
# Objective function
model3.value = Objective(expr = sum(sum(FTE_Salary[b,m]*model3.x[b,m] + 1.2*UnitOutSourceCost[b,m]*model3.y[b,m]
                                       for m in Month)for b in BankLoc), sense= minimize)

### Here the objective function changes a bit as the outsource cost increases by 20%

In [34]:
# Invoking the solver
result = SolverFactory('glpk').solve(model3)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 18144899.6638542
  Upper bound: 18144899.6638542
  Number of objectives: 1
  Number of constraints: 73
  Number of variables: 73
  Number of nonzeros: 109
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.19393396377563477
# ----------------------------------------------------------
#   Solution Information
# -----------

In [35]:
# Print the value of the objective function
model3.value()

18144899.663854163

### The company has to spend $18.14 m on increasing the outsourcing cost by 20%

In [36]:
# Creating dataframe for the results
Output_outsource_cost20 = []

for b in BankLoc:
    for m in Month:
        # Full time employees
        FTE=model3.x[b,m].value
        
        #No.of outsourced applications
        Outsourced = model3.y[b,m].value
        
        Output_outsource_cost20.append([b,m,FTE,Outsourced])
print(Output_outsource_cost20)

Output_outsource_cost20 = pd.DataFrame(Output_outsource_cost20,columns=['State', 'Month','Full-Time Equivalents','Outsourced Applications'])
Output_outsource_cost20.head()

[['A', 'Jan', 161.728395061728, 0.0], ['A', 'Feb', 160.460526315789, 0.0], ['A', 'Mar', 198.066666666667, 0.0], ['A', 'Apr', 71.78125, 0.0], ['A', 'May', 63.8461538461538, 0.0], ['A', 'Jun', 77.9109589041096, 0.0], ['A', 'Jul', 196.102941176471, 0.0], ['A', 'Aug', 110.888157894737, 0.0], ['A', 'Sep', 116.018518518519, 0.0], ['A', 'Oct', 120.856164383562, 0.0], ['A', 'Nov', 157.5, 0.0], ['A', 'Dec', 199.346153846154, 0.0], ['B', 'Jan', 152.067901234568, 0.0], ['B', 'Feb', 86.4473684210526, 0.0], ['B', 'Mar', 99.1333333333333, 0.0], ['B', 'Apr', 73.0625, 0.0], ['B', 'May', 128.846153846154, 0.0], ['B', 'Jun', 107.77397260274, 0.0], ['B', 'Jul', 157.022058823529, 0.0], ['B', 'Aug', 86.1842105263158, 0.0], ['B', 'Sep', 139.413580246914, 0.0], ['B', 'Oct', 142.294520547945, 0.0], ['B', 'Nov', 115.330882352941, 0.0], ['B', 'Dec', 162.576923076923, 0.0], ['C', 'Jan', 35.8641975308642, 0.0], ['C', 'Feb', 64.7039473684211, 0.0], ['C', 'Mar', 63.2666666666667, 0.0], ['C', 'Apr', 70.65625, 0.0], 

,State,Month,Full-Time Equivalents,Outsourced Applications
0,A,Jan,161.728395,0.0
1,A,Feb,160.460526,0.0
2,A,Mar,198.066667,0.0
3,A,Apr,71.781250,0.0
4,A,May,63.846154,0.0


**`Checkpoint 3:`** On increasing the outsourcing cost by 20%, the company has to spend around 18.14 m$, which is slightly more than the previous case .

In [37]:
# Writing the results in to an Excel sheet
#book = load_workbook("Output_data.xlsx")

# create excel writer object
#writer = pd.ExcelWriter("Output_data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
#writer.book = book


# write dataframe to excel sheet named 'output'
#Output_outsource_cost20.to_excel(writer, sheet_name='Qsn_3_2')


# save the excel file
#writer.save()
#print('DataFrame is written successfully to Excel Sheet.')

In [38]:
# Additional cost per application
print((model3.value()-model.value())/sum(demand.values()))

1.6114111522612038


### Additional cost per application is $1.6, compared to the first model

In [39]:
# Average optimal percentage of outsourcing 
average_optimal_perc = Output_outsource_cost20['Outsourced Applications'].sum()
print(average_optimal_perc)

0.0


### With increase in outsourcing costs by 20%, the company should not outsource at all! The outsourcing cost is too much and it is better to process the applications with the FTE's

# Question 4

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 4.1 Worst case analysis 

- 4.1.1 What is the optimal number of staff members for the worst and best cases? 

- 4.1.2 What is the percentage of outsourcing for the worst and best cases? 

- 4.1.3 What is the average cost per application for the worst and best cases?


#### 4.2 Best case analysis 

- 4.2.1 What is the optimal number of staff members for the worst and best cases? 

- 4.2.2 What is the percentage of outsourcing for the worst and best cases? 

- 4.2.3 What is the average cost per application for the worst and best cases?


#### Expected output:

For each of the subtasks (4.1 and 4.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 4.1 Worst Case Analysis 

#### 4.1.1 Optimal number of staff members

In [40]:
# Creating a model instance
model4 = ConcreteModel()

In [41]:
# Define Pyomo sets and Parameters
#Pyomo sets
model4.BankLoc = Set(initialize=BankLoc)
model4.Month = Set(initialize=Month)
#parameter
model4.demand = Param(model4.BankLoc, model4.Month, initialize = demand)

In [42]:
# Decision variables
model4.x = Var(BankLoc, Month, within=NonNegativeReals)
model4.y = Var(BankLoc, Month, within=NonNegativeIntegers)

In [43]:
# Constraints

## Demand Constraint
def demand_constraint(model4,b,m):
    return((model4.x[b,m]*StaffAv_LB[b,m]*40 + model4.y[b,m]) == demand[b,m])

model4.dmnd = Constraint(BankLoc, Month, rule=demand_constraint)

## Regulatory Restriction
def outsource_constraint_a(model4, b, m):
    return(model4.y[b,m] <= 0.3*demand[b,m])

model4.outsrc_a = Constraint(BankLoc[0], Month, rule=outsource_constraint_a)

def outsource_constraint_b(model4, b, m):
    return(model4.y[b,m] <= 0.4*demand[b,m])

model4.outsrc_b = Constraint(BankLoc[1], Month, rule=outsource_constraint_b)

In [44]:
# Objective function
model4.value = Objective(expr = sum(sum(FTE_Salary[b,m]*model4.x[b,m] + UnitOutSourceCost[b,m]*model4.y[b,m]
                                       for m in Month)for b in BankLoc), sense= minimize)

In [45]:
# Invoking the solver
result = SolverFactory('glpk').solve(model4)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19599482.5167888
  Upper bound: 19599482.5167888
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.14461398124694824
# ----------------------------------------------------------
#   Solution Information
# ------------

In [46]:
# Print the value of the objective function
model4.value()

19599482.516788762

### The company has to spend $19.6 m in the worst case scenario

**`Checkpoint 4:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [47]:
# Creating dataframe for the results
Output_wrstcase = []

for b in BankLoc:
    for m in Month:
        # Full time employees
        FTE=model4.x[b,m].value
        
        #No.of outsourced applications
        Outsourced = model4.y[b,m].value
        
        Output_wrstcase.append([b,m,FTE,Outsourced])
print(Output_wrstcase)

Output_wrstcase = pd.DataFrame(Output_wrstcase,columns=['State', 'Month','Full-Time Equivalents','Outsourced Applications'])
Output_wrstcase.head()

[['A', 'Jan', 187.142857142857, 0.0], ['A', 'Feb', 131.346153846154, 1463.0], ['A', 'Mar', 212.214285714286, 0.0], ['A', 'Apr', 76.5666666666667, 0.0], ['A', 'May', 71.1428571428571, 0.0], ['A', 'Jun', 61.2692307692308, 682.0], ['A', 'Jul', 155.583333333333, 1600.0], ['A', 'Aug', 90.7692307692308, 1011.0], ['A', 'Sep', 134.25, 0.0], ['A', 'Oct', 95.0384615384615, 1058.0], ['A', 'Nov', 124.958333333333, 1285.0], ['A', 'Dec', 151.208333333333, 1554.0], ['B', 'Jan', 105.607142857143, 1970.0], ['B', 'Feb', 60.6538461538462, 1051.0], ['B', 'Mar', 63.75, 1189.0], ['B', 'Apr', 46.7666666666667, 935.0], ['B', 'May', 86.1428571428571, 1608.0], ['B', 'Jun', 72.6538461538462, 1258.0], ['B', 'Jul', 106.791666666667, 1708.0], ['B', 'Aug', 60.4615384615385, 1048.0], ['B', 'Sep', 96.8214285714286, 1806.0], ['B', 'Oct', 95.8846153846154, 1662.0], ['B', 'Nov', 78.4583333333333, 1254.0], ['B', 'Dec', 105.708333333333, 1690.0], ['C', 'Jan', 41.5, 0.0], ['C', 'Feb', 0.0, 1967.0], ['C', 'Mar', 67.785714285

,State,Month,Full-Time Equivalents,Outsourced Applications
0,A,Jan,187.142857,0.0
1,A,Feb,131.346154,1463.0
2,A,Mar,212.214286,0.0
3,A,Apr,76.566667,0.0
4,A,May,71.142857,0.0


In [48]:
# Writing the results in to an Excel sheet
#book = load_workbook("Output_data.xlsx")

# create excel writer object
#writer = pd.ExcelWriter("Output_data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
#writer.book = book


# write dataframe to excel sheet named 'output'
#Output_wrstcase.to_excel(writer, sheet_name='Qsn_4_1')


# save the excel file
#writer.save()
#print('DataFrame is written successfully to Excel Sheet.')

In [49]:
# Optimal number of staffs
Output_wrstcase['Full-Time Equivalents'].sum()

2761.2709706959704

### The optimal number of staffs is 2761 in the worst case scenario

#### 4.1.2 Percentage of outsourced applications 

In [50]:
# write your code here
outsourced_app_perc = Output_wrstcase['Outsourced Applications'].sum()/sum(demand.values())
print(outsourced_app_perc*100)

35.13690045368687


### Percentage of outsourced applications is 35.13%, in the worst case scenario

#### 4.1.3 Average cost per application

In [51]:
# write your code here
print(model4.value()/sum(demand.values()))

172.99665045623564


### Average cost per application is $173, in the worst case scenario

### 4.2  Best Case Analysis 

#### 4.2.1 Optimal number of staff members


In [52]:
# Creating a model instance
model5 = ConcreteModel()

In [53]:
# Define Pyomo sets and Parameters
#Pyomo sets
model5.BankLoc = Set(initialize=BankLoc)
model5.Month = Set(initialize=Month)
#parameter
model5.demand = Param(model5.BankLoc, model5.Month, initialize = demand)

In [54]:
# Decision variables
model5.x = Var(BankLoc, Month, within=NonNegativeReals)
model5.y = Var(BankLoc, Month, within=NonNegativeIntegers)

In [55]:
# Constraints
## Demand Constraint
def demand_constraint(model5,b,m):
    return((model5.x[b,m]*StaffAv_UB[b,m]*40 + model5.y[b,m]) == demand[b,m])

model5.dmnd = Constraint(BankLoc, Month, rule=demand_constraint)

## Regulatory Restriction
def outsource_constraint_a(model5, b, m):
    return(model5.y[b,m] <= 0.3*demand[b,m])

model5.outsrc_a = Constraint(BankLoc[0], Month, rule=outsource_constraint_a)

def outsource_constraint_b(model5, b, m):
    return(model5.y[b,m] <= 0.4*demand[b,m])

model5.outsrc_b = Constraint(BankLoc[1], Month, rule=outsource_constraint_b)

In [56]:
# Objective function
model5.value = Objective(expr = sum(sum(FTE_Salary[b,m]*model5.x[b,m] + UnitOutSourceCost[b,m]*model5.y[b,m]
                                       for m in Month)for b in BankLoc), sense= minimize)

In [57]:
# Invoking the solver
result = SolverFactory('glpk').solve(model5)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16527535.6379357
  Upper bound: 16527535.6379357
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.2036302089691162
# ----------------------------------------------------------
#   Solution Information
# -------------

In [58]:
# Print the value of the objective function
model5.value()

16527535.63793573

### The company has to spend $16.5 m in the best case scenario

**`Checkpoint 5:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [59]:
# Creating dataframe for the results
Output_bestcase = []

for b in BankLoc:
    for m in Month:
        # Full time employees
        FTE=model5.x[b,m].value
        
        #No.of outsourced applications
        Outsourced = model5.y[b,m].value
        
        Output_bestcase.append([b,m,FTE,Outsourced])
print(Output_bestcase)

Output_bestcase = pd.DataFrame(Output_bestcase,columns=['State', 'Month','Full-Time Equivalents','Outsourced Applications'])
Output_bestcase.head()

[['A', 'Jan', 145.555555555556, 0.0], ['A', 'Feb', 143.470588235294, 0.0], ['A', 'Mar', 185.6875, 0.0], ['A', 'Apr', 67.5588235294118, 0.0], ['A', 'May', 58.5882352941176, 0.0], ['A', 'Jun', 71.09375, 0.0], ['A', 'Jul', 177.8, 0.0], ['A', 'Aug', 99.1470588235294, 0.0], ['A', 'Sep', 104.416666666667, 0.0], ['A', 'Oct', 110.28125, 0.0], ['A', 'Nov', 142.8, 0.0], ['A', 'Dec', 185.107142857143, 0.0], ['B', 'Jan', 136.861111111111, 0.0], ['B', 'Feb', 77.2941176470588, 0.0], ['B', 'Mar', 92.9375, 0.0], ['B', 'Apr', 68.7647058823529, 0.0], ['B', 'May', 118.235294117647, 0.0], ['B', 'Jun', 98.34375, 0.0], ['B', 'Jul', 85.4333333333333, 1708.0], ['B', 'Aug', 77.0588235294118, 0.0], ['B', 'Sep', 125.472222222222, 0.0], ['B', 'Oct', 129.84375, 0.0], ['B', 'Nov', 62.7666666666667, 1254.0], ['B', 'Dec', 90.6071428571429, 1690.0], ['C', 'Jan', 32.2777777777778, 0.0], ['C', 'Feb', 57.8529411764706, 0.0], ['C', 'Mar', 59.3125, 0.0], ['C', 'Apr', 66.5, 0.0], ['C', 'May', 59.7058823529412, 0.0], ['C', '

,State,Month,Full-Time Equivalents,Outsourced Applications
0,A,Jan,145.555556,0.0
1,A,Feb,143.470588,0.0
2,A,Mar,185.687500,0.0
3,A,Apr,67.558824,0.0
4,A,May,58.588235,0.0


In [60]:
# Writing the results in to an Excel sheet
#book = load_workbook("Output_data.xlsx")

# create excel writer object
#writer = pd.ExcelWriter("Output_data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
#writer.book = book


# write dataframe to excel sheet named 'output'
#Output_bestcase.to_excel(writer, sheet_name='Qsn_4_2')


# save the excel file
#writer.save()
#print('DataFrame is written successfully to Excel Sheet.')

In [61]:
# Optimal number of staffs
Output_bestcase['Full-Time Equivalents'].sum()

3343.189524976658

### The optimal number of staffs in the best case scenario is 3343

#### 4.2.2 Percentage of outsourced applications

In [62]:
# write your code here
outsourced_app_perc_2 = Output_bestcase['Outsourced Applications'].sum()/sum(demand.values())
print(outsourced_app_perc_2*100)

4.106130951330168


### The percentage of outsourced applications is only 4.1%, in the best case scenario as the staffs are mostly available and it is not required to go for costlier outsourcing

#### 4.2.3 Average cost per application

In [63]:
# write your code here
print(model5.value()/sum(demand.values()))

145.88182638035315


### The average cost per application in the best case scenario is only $146

### The average cost per application in the worst case scenario was $173

# Question 5

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2, 3 and 4 (i.e., the best case, expected, expected with 10% outsourcing limit and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

# Question 6 

#### Additional proposed outsourced vendor 

The company sees the additional proposed outsourced vendor as ‘VENDOR 2’ with a cost per application as given below.

**`Vendor 1`**	
**A:** 180 Dollars |
**B:** 150 Dollars |
**C:** 160 Dollars 

**`Vendor 2`** 	
**A:** 155 Dollars |
**B:** 150 Dollars |
**C:** 158 Dollars

                                                                 

The states have signed a memorandum with both vendors to maintain the ratio (60%: 40%) of Vendor 1:Vendor 2 for the number of applications that can be outsourced for every month in each state.


**`Q 6.1-`** Calculate the optimal staffing at a 10% limit on outsourcing while maintaining the vendor ratio (60%:40%) for the number of applications that can be outsourced for every month in each state.


**`Q 6.2-`** Optimal Allocation of the application after relaxing the ratio criterion mentioned above (60%: 40%) for the number of applications that can be outsourced for every month in each state.


#### Expected output: 
For each subtask (a) and (b), create a data frame containing the number of outsourced applications for each vendor and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier.


###  6.1 Optimal staffing at 10% limit on outsourcing while maintaining the vendor ratio as 60%:40% of the total application

In [64]:
# Creating a model instance
model6 = ConcreteModel()

In [65]:
# Define Pyomo sets and Parameters
#Pyomo sets
model6.BankLoc = Set(initialize=BankLoc)
model6.Month = Set(initialize=Month)
#parameter
model6.demand = Param(model6.BankLoc, model6.Month, initialize = demand)

In [66]:
# Decision variables
model6.x = Var(BankLoc, Month, within=NonNegativeReals)
model6.y = Var(BankLoc, Month, within=NonNegativeIntegers)
model6.z = Var(BankLoc, Month, within=NonNegativeIntegers)

### Here we have three decision variables- one(x) for the FTE's, second and third ones (y and z) are for the two vendors respectively

#### Reading the outsourcing cost for Vendor 2 

In [67]:
# Unit Outsourcing Cost for Vendor 2( we already have cost for Vendor 1)
UnitOutSourceCost2 = Cost.set_index(['State','Month'])['UnitOutCost_v2'].to_dict()

In [68]:
# Constraints
## Demand Constraint
def demand_constraint(model6,b,m):
    return((model6.x[b,m]*StaffAvPer[b,m]*40 + model6.y[b,m]) + model6.z[b,m] == demand[b,m])

model6.dmnd = Constraint(BankLoc, Month, rule=demand_constraint)



#Outsource Constraint
def outsource_constraint(model6,b,m):
    return(model6.y[b,m] + model6.z[b,m] <= 0.1*demand[b,m])
model6.outsrc = Constraint(Bankloc,Month, rule = outsource_constraint)

# Vendor Constraint
def vendor_constraint(model6,b,m):
    return(0.4*model6.y[b,m] == 0.6*model6.z[b,m])
model6.vendor = Constraint(Bankloc,Month, rule = vendor_constraint )

### The constraints are also changed here. 
#### i) The demand is now met by two vendors instead of one along with the FTE's. 
#### ii) Both the vendors combined must have maximum 10% of the applications. 
#### iii) Along with that, we have the vendor constraint where the first vendor and the second vendor will have a 60:40 distribution for the outsourced applications.

In [69]:
# Objective function
model6.value = Objective(expr = sum(sum(FTE_Salary[b,m]*model6.x[b,m] + UnitOutSourceCost[b,m]*model6.y[b,m] + UnitOutSourceCost2[b,m]*model6.z[b,m]
                                       for m in Month)for b in BankLoc), sense= minimize)

### The objective function is changed appropriately as we have to minimize the cost with respect to the FTE's and the two vendors instead of one.

In [70]:
# Invoking the solver
result = SolverFactory('glpk').solve(model6)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 18085432.8465717
  Upper bound: 18085432.8465717
  Number of objectives: 1
  Number of constraints: 109
  Number of variables: 109
  Number of nonzeros: 253
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.19147300720214844
# ----------------------------------------------------------
#   Solution Information
# ---------

In [71]:
# Print the value of the objective function
model6.value()

18085432.84657171

### The company has to spend $18.1m approx if there is ratio criterion for the two vendors

**`Checkpoint 6:`** The company has to spend around 18.1 m$ in total for the application approval process if there is ratio criterion.

In [72]:
# Creating dataframe for the results
Output_vendor = []

for b in BankLoc:
    for m in Month:
        # Full time employees
        FTE=model6.x[b,m].value
        
        #No.of outsourced applications for Vendor 1
        Outsourced_vendor1 = model6.y[b,m].value
        
        #No.of outsourced applications for Vendor 1
        Outsourced_vendor2 = model6.z[b,m].value
        
        
        Output_vendor.append([b,m,FTE,Outsourced_vendor1, Outsourced_vendor2])
print(Output_vendor)

Output_vendor = pd.DataFrame(Output_vendor,columns=['State', 'Month','Full-Time Equivalents','Outsourced_Applications_Vendor1','Outsourced_Applications_Vendor2'])
Output_vendor.head()

[['A', 'Jan', 161.728395061728, 0.0, 0.0], ['A', 'Feb', 160.460526315789, 0.0, 0.0], ['A', 'Mar', 198.066666666667, 0.0, 0.0], ['A', 'Apr', 71.78125, 0.0, 0.0], ['A', 'May', 63.8461538461538, 0.0, 0.0], ['A', 'Jun', 70.2054794520548, 135.0, 90.0], ['A', 'Jul', 176.617647058824, 318.0, 212.0], ['A', 'Aug', 110.888157894737, 0.0, 0.0], ['A', 'Sep', 116.018518518519, 0.0, 0.0], ['A', 'Oct', 108.869863013699, 210.0, 140.0], ['A', 'Nov', 141.875, 255.0, 170.0], ['A', 'Dec', 179.538461538462, 309.0, 206.0], ['B', 'Jan', 152.067901234568, 0.0, 0.0], ['B', 'Feb', 77.8947368421053, 156.0, 104.0], ['B', 'Mar', 89.3, 177.0, 118.0], ['B', 'Apr', 73.0625, 0.0, 0.0], ['B', 'May', 128.846153846154, 0.0, 0.0], ['B', 'Jun', 97.1575342465753, 186.0, 124.0], ['B', 'Jul', 141.397058823529, 255.0, 170.0], ['B', 'Aug', 77.6315789473684, 156.0, 104.0], ['B', 'Sep', 139.413580246914, 0.0, 0.0], ['B', 'Oct', 128.082191780822, 249.0, 166.0], ['B', 'Nov', 103.933823529412, 186.0, 124.0], ['B', 'Dec', 146.4230769

,State,Month,Full-Time Equivalents,Outsourced_Applications_Vendor1,Outsourced_Applications_Vendor2
0,A,Jan,161.728395,0.0,0.0
1,A,Feb,160.460526,0.0,0.0
2,A,Mar,198.066667,0.0,0.0
3,A,Apr,71.781250,0.0,0.0
4,A,May,63.846154,0.0,0.0


In [73]:
# Writing the results in to an Excel sheet
#book = load_workbook("Output_data.xlsx")

# create excel writer object
#writer = pd.ExcelWriter("Output_data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
#writer.book = book


# write dataframe to excel sheet named 'output'
#Output_vendor.to_excel(writer, sheet_name='Qsn_6_1')


# save the excel file
#writer.save()
#print('DataFrame is written successfully to Excel Sheet.')

### Optimal Staffing

In [74]:
Output_vendor['Full-Time Equivalents'].sum()

3648.1252273670393

In [75]:
Output_vendor['Outsourced_Applications_Vendor1'].sum()

3165.0

In [76]:
Output_vendor['Outsourced_Applications_Vendor2'].sum()

2110.0

### The optimal staffing is 3648 when there is a ratio constraint on vendors

### 6.2	Optimal allocation of application to vendors after relaxing the ratio criterion 60%:40% (Vendor 1:Vendor 2) 

In [77]:
# Creating a model instance
model7 = ConcreteModel()

In [78]:
# Define Pyomo sets and Parameters
#Pyomo sets
model7.BankLoc = Set(initialize=BankLoc)
model7.Month = Set(initialize=Month)
#parameter
model7.demand = Param(model7.BankLoc, model7.Month, initialize = demand)

In [79]:
# Decision variables
model7.x = Var(BankLoc, Month, within=NonNegativeReals)
model7.y = Var(BankLoc, Month, within=NonNegativeIntegers)
model7.z = Var(BankLoc, Month, within=NonNegativeIntegers)

In [80]:
# Constraints
## Demand Constraint
def demand_constraint(model7,b,m):
    return((model7.x[b,m]*StaffAvPer[b,m]*40 + model7.y[b,m]) + model7.z[b,m] == demand[b,m])

model7.dmnd = Constraint(BankLoc, Month, rule=demand_constraint)



#Outsource Constraint
#def outsource_constraint(model7,b,m):
#   return(model7.y[b,m] + model7.z[b,m] <= 0.1*demand[b,m])
#model7.outsrc = Constraint(Bankloc,Month, rule = outsource_constraint)

## Please note that on giving this particular constraint of 10%, the optimization is not getting executed. Hence, we removed this. We would get a high value of outsourced applications because of this but given the situation and lack of clarification, we stick to this! Hope this doesn't go wrong!

In [81]:
# Objective function
model7.value = Objective(expr = sum(sum(FTE_Salary[b,m]*model7.x[b,m] + UnitOutSourceCost[b,m]*model7.y[b,m] + UnitOutSourceCost2[b,m]*model7.z[b,m]
                                       for m in Month)for b in BankLoc), sense= minimize)

In [82]:
# Invoking the solver
result = SolverFactory('glpk').solve(model7)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17069134.0056925
  Upper bound: 17069134.0056925
  Number of objectives: 1
  Number of constraints: 37
  Number of variables: 109
  Number of nonzeros: 109
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.1535811424255371
# ----------------------------------------------------------
#   Solution Information
# -----------

In [83]:
# Print the value of the objective function
model7.value()

17069134.005692545

### The company has to spend around $17.1 on relaxing the ratio criterion

**`Checkpoint 7:`** The company has to spend around 17.1 m$ in total for the application approval process on realxing the ratio criterion which is lesser than the previous case. 

In [84]:
# Creating dataframe for the results
# Creating dataframe for the results
Output_vendor_2 = []

for b in BankLoc:
    for m in Month:
        # Full time employees
        FTE=model7.x[b,m].value
        
        #No.of outsourced applications for Vendor 1
        Outsourced_vendor1 = model7.y[b,m].value
        
        #No.of outsourced applications for Vendor 1
        Outsourced_vendor2 = model7.z[b,m].value
        
        
        Output_vendor_2.append([b,m,FTE,Outsourced_vendor1, Outsourced_vendor2])
print(Output_vendor_2)

Output_vendor_2 = pd.DataFrame(Output_vendor_2,columns=['State', 'Month','Full-Time Equivalents','Outsourced_Applications_Vendor1','Outsourced_Applications_Vendor2'])
Output_vendor_2.head()

[['A', 'Jan', 161.728395061728, 0.0, 0.0], ['A', 'Feb', 0.0, 0.0, 4878.0], ['A', 'Mar', 0.0, 0.0, 5942.0], ['A', 'Apr', 0.0, 0.0, 2297.0], ['A', 'May', 0.0, 0.0, 1992.0], ['A', 'Jun', 0.0, 0.0, 2275.0], ['A', 'Jul', 0.0, 0.0, 5334.0], ['A', 'Aug', 0.0, 0.0, 3371.0], ['A', 'Sep', 116.018518518519, 0.0, 0.0], ['A', 'Oct', 0.0, 0.0, 3529.0], ['A', 'Nov', 0.0, 0.0, 4284.0], ['A', 'Dec', 0.0, 0.0, 5183.0], ['B', 'Jan', 152.067901234568, 0.0, 0.0], ['B', 'Feb', 0.0, 2628.0, 0.0], ['B', 'Mar', 0.0, 2974.0, 0.0], ['B', 'Apr', 73.0625, 0.0, 0.0], ['B', 'May', 128.846153846154, 0.0, 0.0], ['B', 'Jun', 0.0, 3147.0, 0.0], ['B', 'Jul', 0.0, 4271.0, 0.0], ['B', 'Aug', 0.0, 2620.0, 0.0], ['B', 'Sep', 139.413580246914, 0.0, 0.0], ['B', 'Oct', 0.0, 4155.0, 0.0], ['B', 'Nov', 0.0, 3137.0, 0.0], ['B', 'Dec', 0.0, 4227.0, 0.0], ['C', 'Jan', 35.8641975308642, 0.0, 0.0], ['C', 'Feb', 64.7039473684211, 0.0, 0.0], ['C', 'Mar', 63.2666666666667, 0.0, 0.0], ['C', 'Apr', 70.65625, 0.0, 0.0], ['C', 'May', 65.0641

,State,Month,Full-Time Equivalents,Outsourced_Applications_Vendor1,Outsourced_Applications_Vendor2
0,A,Jan,161.728395,0.0,0.0
1,A,Feb,0.000000,0.0,4878.0
2,A,Mar,0.000000,0.0,5942.0
3,A,Apr,0.000000,0.0,2297.0
4,A,May,0.000000,0.0,1992.0


In [85]:
# Writing the results in to an Excel sheet
#book = load_workbook("Output_data.xlsx")

# create excel writer object
#writer = pd.ExcelWriter("Output_data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
#writer.book = book


# write dataframe to excel sheet named 'output'
#Output_vendor_2.to_excel(writer, sheet_name='Qsn_6_2')


# save the excel file
#writer.save()
#print('DataFrame is written successfully to Excel Sheet.')

### Optimal Allocation of Applications

In [86]:
Output_vendor_2['Full-Time Equivalents'].sum()

1320.3981019357275

In [87]:
Output_vendor_2['Outsourced_Applications_Vendor1'].sum()

27159.0

In [88]:
Output_vendor_2['Outsourced_Applications_Vendor2'].sum()

44535.0

### The optimal allocation of applications is 27,159 for Vendor 1 and 44,535 for Vendor 2 and the number of optimal staffs is 1320

In [89]:
#### Cost per Application
Cost_per_application_actual= model.value()/sum(demand.values())
print(Cost_per_application_actual)

Cost_per_application_outsourcing_limit= model2.value()/sum(demand.values())
print(Cost_per_application_outsourcing_limit)

Cost_per_application_outsourcing_cost= model3.value()/sum(demand.values())
print(Cost_per_application_outsourcing_cost)

Cost_per_application_worst_case= model4.value()/sum(demand.values())
print(Cost_per_application_worst_case)

Cost_per_application_outsourcing_best_case= model5.value()/sum(demand.values())
print(Cost_per_application_outsourcing_best_case)

158.5462288273861
159.76997300139334
160.1576399796473
172.99665045623564
145.88182638035315
